<a href="https://colab.research.google.com/github/giolotar/Turkiye_detect/blob/main/Inference_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Turkey Detect Inference Runner**

**The following code loads a pretrained model with weights and configuration**

The model was trained on the rubble instances of a single sat image of Antakya, Turkyie. Annotations were made in CVAT with one label "Damage" = "Rubble
The model weights and configuration are loaded and assembled into a predictor.

The predictor then ingests a tif image, tiles it, runs predictions on the tiles, based on that predictions it does two things:



1.   Generates a masks file .npz with the bounding boxes, coordinates and confidence grade for each prediction. the file which is then used to make a shapefile that is georeferenced to the injected tif
2.   Generates a simple png image with the printed masks on it



## **SET-UP**

In [1]:
#Mount up the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Install Detectron2 Library
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-v709x1ff
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-v709x1ff
  Resolved https://github.com/facebookresearch/detectron2.git to commit a1ce2f956a1d2212ad672e3c47d53405c2fe4312
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.

In [3]:
#GPU Check

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.8 ; cuda:  cu126
detectron2: 0.6


In [4]:
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# Detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### **Load the Model and Config**

Here we load a pretrained model configuration and weights

In [6]:

cfg_pretrained = get_cfg()

saved_config_path = "/content/drive/MyDrive/colabDL/turkey_OGrun.yaml"
saved_weights_path = "/content/drive/MyDrive/colabDL/tile1/model_final_cfg2.pth"

# Load the configuration directly from your saved .yaml file
try:
    cfg_pretrained.merge_from_file(saved_config_path)
except KeyError as e:
    print(f"Error loading configuration from {saved_config_path}: {e}")
    raise

# Set the path to the model weights
cfg_pretrained.MODEL.WEIGHTS = saved_weights_path

# Ensure the output directory is created (needed for the predictor)
cfg_pretrained.OUTPUT_DIR = "/content/sample_data"
os.makedirs(cfg_pretrained.OUTPUT_DIR, exist_ok=True)

### **Compile Predictor**

In [7]:
# Test Threshold
cfg_pretrained.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

# Assemble predictor
predictor = DefaultPredictor(cfg_pretrained)

print("DefaultPredictor initialized successfully with loaded configuration and weights.")

[10/06 09:24:29 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/colabDL/tile1/model_final_cfg2.pth ...
DefaultPredictor initialized successfully with loaded configuration and weights.


Try to predict on an image / tile see if the predictor works

In [ ]:
new_im = cv2.imread("/content/drive/MyDrive/colabDL/tile1/test_row1to6_8/tile_y4095_x7371_row5_col9.tif")
outputs  = predictor(new_im)

# We can use `Visualizer` to draw the predictions on the image.

#!!! NEED TO EXPLORE WITH VISUALIZER OPTIONS MORE APPEALING MASKS REPRESENTATIONS
v = Visualizer(new_im[:, :, ::-1], metadata=None)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(out.get_image()[:, :, ::-1])

# **Inference**

Then we run the predictor on an unseen Georef image, the image gets tiled, predictions are run on the tiles and exported in .npz as georeferenced polygons.
Then they are turned into a shapefile

### Load georeferenced TIF

In [8]:
!pip install rasterio fiona shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 138.7 MB/s eta 0:00:00


In [9]:
# tile_tiff_save_preds.py
import os
import math
import cv2
import numpy as np
import rasterio # Import rasterio for georeferenced image handling

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog  # needed for Visualizer

# ==== USER PATHS ====
INPUT_TIFF = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif" # Use the georeferenced TIFF
OUT_DIR   = "/content/drive/MyDrive/colabDL/Turkiye/TR1"  # main output directory
NPZ_OUT_DIR = os.path.join(OUT_DIR, "npz_preds") # subdirectory for npz files
TILE = 1024

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(NPZ_OUT_DIR, exist_ok=True)

# Read the georeferenced image and get georeferencing info
try:
    with rasterio.open(INPUT_TIFF) as src:
        img = src.read()
        # Rasterio reads bands first (C, H, W), convert to OpenCV format (H, W, C)
        img = np.transpose(img, (1, 2, 0))
        # Explicitly select the first 3 channels to handle potential alpha channels
        if img.shape[2] == 4:
            img = img[:, :, :3]
        orig_h, orig_w = img.shape[:2]
        transform = src.transform # Affine transform for georeferencing
        crs = src.crs # Coordinate reference system

except rasterio.errors.RasterioIOError as e:
    raise FileNotFoundError(f"Could not read georeferenced image at {INPUT_TIFF}: {e}")

# Pad to multiples of TILE so we don’t lose right/bottom edges
H = math.ceil(orig_h / TILE) * TILE
W = math.ceil(orig_w / TILE) * TILE
pad_bottom = H - orig_h
pad_right  = W - orig_w

if pad_bottom or pad_right:
    # Use the last pixel value for padding to avoid edge effects if possible,
    # otherwise use black padding for color images.
    if img.shape[2] == 3:
        pad_val = [int(img[-1,-1,0]), int(img[-1,-1,1]), int(img[-1,-1,2])]
    else:
        pad_val = [0] * img.shape[2]
    img = cv2.copyMakeBorder(
        img, 0, pad_bottom, 0, pad_right,
        borderType=cv2.BORDER_CONSTANT, value=pad_val
    )

rows = H // TILE
cols = W // TILE

tiles_written = 0
dets_saved = 0

for y in range(rows):
    for x in range(cols):
        r0 = y * TILE
        c0 = x * TILE
        tile = img[r0:r0+TILE, c0:c0+TILE]

        # Calculate georeferenced origin of the tile (top-left corner)
        geo_x0, geo_y0 = transform * (c0, r0)

        # ---- Run inference on the tile
        outputs = predictor(tile)
        inst = outputs["instances"].to("cpu")

        # ---- Extract raw predictions
        boxes  = inst.pred_boxes.tensor.numpy() if inst.has("pred_boxes") else np.zeros((0,4), dtype=np.float32)
        scores = inst.scores.numpy()             if inst.has("scores")     else np.zeros((0,), dtype=np.float32)
        classes= inst.pred_classes.numpy()       if inst.has("pred_classes") else np.zeros((0,), dtype=np.int32)
        masks  = inst.pred_masks.numpy()         if inst.has("pred_masks") else None  # (N, H, W) boolean

        # ---- SAVE RAW PREDICTIONS (compressed)
        # Include tile origin in pixel and georeferenced coordinates
        npz_path = os.path.join(NPZ_OUT_DIR, f"tile_{y}_{x}_pred.npz")
        np.savez_compressed(
            npz_path,
            boxes=boxes,
            scores=scores,
            classes=classes,
            masks=masks,                 # None is allowed; np.load(..., allow_pickle=True) will handle it
            tile_row=y,
            tile_col=x,
            tile_origin_rc=np.array([r0, c0], dtype=np.int32), # Pixel origin (row, col)
            tile_origin_geo=np.array([geo_x0, geo_y0], dtype=np.float64), # Georeferenced origin (x, y)
            tile_size=TILE,
            padded_h=H,
            padded_w=W,
            orig_h=orig_h,
            orig_w=orig_w,
            pad_bottom=pad_bottom,
            pad_right=pad_right
        )
        dets_saved += len(boxes)

print(f"Done. Wrote {tiles_written} visualized tiles to {OUT_DIR}")
print(f"Saved predictions for {rows*cols} tiles ({dets_saved} detections total) as .npz files to {NPZ_OUT_DIR}")

/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Done. Wrote 0 visualized tiles to /content/drive/MyDrive/colabDL/Turkiye/TR1
Saved predictions for 80 tiles (157 detections total) as .npz files to /content/drive/MyDrive/colabDL/Turkiye/TR1/npz_preds


convert from the npz to shp retaining geographic coordinates

In [10]:
import os
import numpy as np
import re
import rasterio
from shapely.geometry import Polygon, mapping
import fiona

In [11]:
# Directory containing the predicted tile .npz files
PREDICTED_NPZ_DIR = "/content/drive/MyDrive/colabDL/Turkiye/TR1/npz_preds" # Assuming npz files are in a subdirectory

# Output shapefile/geopackage path
# Changed to .gpkg for geopackage, which is preferred
OUTPUT_VECTOR_PATH = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.gpkg"

# Path to the original georeferenced image (GeoTIFF or similar)

# This is necessary to get the coordinate system and transform information.
ORIGINAL_GEOREFERENCED_IMAGE_PATH = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif" # Example path, replace with your actual georeferenced image path

# --- Function to get georeferencing information ---
def get_georeferencing_info(image_path):
    """Reads georeferencing information (transform and CRS) from a georeferenced image."""
    try:
        with rasterio.open(image_path) as src:
            return src.transform, src.crs
    except rasterio.errors.RasterioIOError as e:
        print(f"Error reading georeferencing info from {image_path}: {e}")
        print("Assuming pixel coordinates. The output vector file will NOT be georeferenced.")
        return None, None

# Get georeferencing info from the original image
transform, crs = get_georeferencing_info(ORIGINAL_GEOREFERENCED_IMAGE_PATH)

# --- Define the schema for the vector file ---
# This defines the attributes (columns) for each feature (polygon) in the vector file.
# We'll include score and class_id from the predictions.
schema = {
    'geometry': 'Polygon',
    'properties': {'score': 'float', 'class_id': 'int'},
}

# --- Create the vector file (Geopackage) ---
# Determine the driver based on the output file extension
# Changed driver to GPKG for Geopackage
driver = 'GPKG'

# Open the vector file in write mode
# If a CRS was obtained, use it. Otherwise, the vector file will have no defined CRS.
with fiona.open(
    OUTPUT_VECTOR_PATH,
    'w',
    driver,
    schema,
    crs=crs # Use the obtained CRS, or None if not available
) as collection:
    # List all .npz files in the directory
    npz_filenames = [f for f in os.listdir(PREDICTED_NPZ_DIR) if f.endswith('_pred.npz')]

    if not npz_filenames:
        print("No predicted .npz files found in the directory.")
    else:
        # Iterate through each .npz file
        for filename in npz_filenames:
            npz_path = os.path.join(PREDICTED_NPZ_DIR, filename)
            with np.load(npz_path, allow_pickle=True) as data:
                masks = data['masks']
                scores = data['scores']
                classes = data['classes']
                tile_origin_rc = data['tile_origin_rc'] # Pixel origin (r0, c0)
                # tile_origin_geo = data['tile_origin_geo'] # Georeferenced origin (x0, y0) - not directly used here but available

                if masks is not None and masks.shape[0] > 0:
                    # Process each instance mask within the tile
                    for i in range(masks.shape[0]):
                        instance_mask = masks[i]
                        score = scores[i]
                        class_id = classes[i]

                        # Find contours of the mask
                        # cv2.findContours requires a single-channel binary image
                        # Ensure instance_mask is uint8 and binary (0 or 255)
                        mask_uint8 = instance_mask.astype(np.uint8) * 255
                        # Use cv2.RETR_TREE and cv2.CHAIN_APPROX_NONE for more detailed contours if needed,
                        # but SIMPLE is usually sufficient for polygons.
                        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        # Convert contours to polygons and transform coordinates
                        for contour in contours:
                            # Reshape contour to be a list of points (x, y)
                            # Contour points are in tile coordinates (pixel within the tile)
                            # Ensure points are in (x, y) format which corresponds to (column, row)
                            contour_points_tile = contour.squeeze().tolist()

                            if not isinstance(contour_points_tile, list):
                                # Handle cases where contour.squeeze() results in a single point
                                contour_points_tile = [contour_points_tile]
                            elif not all(isinstance(p, list) for p in contour_points_tile):
                                # Handle cases where contour.squeeze() results in a flat list of coords
                                contour_points_tile = [contour_points_tile]


                            if len(contour_points_tile) < 3:
                                continue # Skip if not enough points to form a polygon

                            # Convert tile pixel coordinates to padded image pixel coordinates
                            # Tile origin is (row, col), contour points are (col, row)
                            contour_points_padded = [(p[0] + tile_origin_rc[1], p[1] + tile_origin_rc[0]) for p in contour_points_tile]

                            # Convert padded image pixel coordinates to geographic coordinates using the original image's transform
                            if transform:
                                # Apply the affine transform to each point
                                # transform * (col, row) gives (x, y) georeferenced coordinates
                                contour_points_geo = [transform * (p[0], p[1]) for p in contour_points_padded]
                                polygon = Polygon(contour_points_geo)
                            else:
                                # If no transform, use padded pixel coordinates for the polygon
                                polygon = Polygon(contour_points_padded)




                                # Add the polygon and its properties to the vector file
                            if not polygon.is_empty and polygon.exterior: # Ensure polygon is valid and not empty
                                # Cast score to a standard float type to potentially avoid Fiona warning
                                collection.write({
                                    'geometry': mapping(polygon),
                                    'properties': {'score': float(score), 'class_id': int(class_id)},
                                })

print(f"Vector file (Geopackage) saved to {OUTPUT_VECTOR_PATH}")

Vector file (Geopackage) saved to /content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.gpkg


Produce a .png image mosaic with the predictions and bounding boxes printed on the image.

In [12]:
import os
import numpy as np
import cv2
import rasterio
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import detectron2.structures
import torch # Import torch for tensor operations if masks are included
import math # Import math for ceil


In [13]:
# ==== USER PATHS ====
INPUT_TIFF = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif" # Use the georeferenced TIFF
OUT_DIR   = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.png"  # Output directory for predicted tile PNGs
TILE = 1024

os.makedirs(OUT_DIR, exist_ok=True)

# Read the georeferenced image and get georeferencing info
try:
    with rasterio.open(INPUT_TIFF) as src:
        img = src.read()
        # Rasterio reads bands first (C, H, W), convert to OpenCV format (H, W, C)
        img = np.transpose(img, (1, 2, 0))
        # Explicitly select the first 3 channels to handle potential alpha channels
        if img.shape[2] == 4:
            img = img[:, :, :3]
        orig_h, orig_w = img.shape[:2]
        transform = src.transform # Affine transform for georeferencing
        crs = src.crs # Coordinate reference system

except rasterio.errors.RasterioIOError as e:
    raise FileNotFoundError(f"Could not read georeferenced image at {INPUT_TIFF}: {e}")

# Pad to multiples of TILE so we don’t lose right/bottom edges
H = math.ceil(orig_h / TILE) * TILE
W = math.ceil(orig_w / TILE) * TILE
pad_bottom = H - orig_h
pad_right  = W - orig_w

if pad_bottom > 0 or pad_right > 0:
    # Use the last pixel value for padding to avoid edge effects if possible,
    # otherwise use black padding for color images.
    if img.shape[2] == 3:
        pad_val = [int(img[-1,-1,0]), int(img[-1,-1,1]), int(img[-1,-1,2])]
    else:
        pad_val = [0] * img.shape[2]
    img_padded = cv2.copyMakeBorder(
        img, 0, pad_bottom, 0, pad_right,
        borderType=cv2.BORDER_CONSTANT, value=pad_val
    )
else:
    img_padded = img

rows = H // TILE
cols = W // TILE

tiles_written = 0

# Use cfg_pretrained to get the dataset metadata for visualization
# Check if DATASETS.TRAIN is defined and not empty
if 'cfg_pretrained' in globals() and hasattr(cfg_pretrained.DATASETS, 'TRAIN') and cfg_pretrained.DATASETS.TRAIN:
    metadata = MetadataCatalog.get(cfg_pretrained.DATASETS.TRAIN[0])
else:
    metadata = None # Use default metadata if not available


for y in range(rows):
    for x in range(cols):
        r0 = y * TILE
        c0 = x * TILE
        tile = img_padded[r0:r0+TILE, c0:c0+TILE]

        # Make prediction on the tile
        outputs = predictor(tile)

        # Visualize the predictions on the tile
        # Convert tile to BGR for Visualizer
        v = Visualizer(tile[:, :, ::-1], metadata=metadata, scale=1.0)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        predicted_tile = out.get_image()[:, :, ::-1] # Convert back to RGB for saving as PNG

        # Save the predicted tile
        cv2.imwrite(os.path.join(OUT_DIR, f"tile_{y}_{x}_pred.png"), predicted_tile)
        tiles_written += 1

print(f"Done. Wrote {tiles_written} predicted tiles to {OUT_DIR}")

Done. Wrote 80 predicted tiles to /content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.png


In [14]:
import os
import cv2
import numpy as np
import re

# Directory containing the predicted tiles
PREDICTED_TILES_DIR = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.png"
# Output path for the remosaiced image
REMOsaICED_OUTPUT_PATH = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_mosaic_1.png"
TILE_SIZE = 1024 # The size of each tile (should match the TILE size used for tiling)

# Get all the predicted tile filenames
tile_filenames = [f for f in os.listdir(PREDICTED_TILES_DIR) if f.endswith('_pred.png')]

if not tile_filenames:
    print("No predicted tile images found in the directory.")
else:
    # Extract rows and columns from filenames to determine the grid size
    max_row = 0
    max_col = 0
    tile_info = {}
    for filename in tile_filenames:
        match = re.match(r'tile_(\d+)_(\d+)_pred\.png', filename)
        if match:
            row = int(match.group(1))
            col = int(match.group(2))
            tile_info[(row, col)] = filename
            max_row = max(max_row, row)
            max_col = max(max_col, col)

    rows = max_row + 1
    cols = max_col + 1

    # Determine the dimensions of the remosaiced image
    # Read one tile to get image dimensions (assuming all tiles have the same dimensions)
    sample_tile_path = os.path.join(PREDICTED_TILES_DIR, tile_filenames[0])
    sample_tile = cv2.imread(sample_tile_path)
    tile_h, tile_w, tile_c = sample_tile.shape

    # Calculate the full image dimensions, considering potential padding
    # The padded dimensions should match the H and W calculated during tiling
    # However, if we saved the predicted tiles without removing the padding visualization,
    # the size will be a multiple of TILE_SIZE.
    full_h = rows * TILE_SIZE
    full_w = cols * TILE_SIZE

    # Create a blank canvas for the remosaiced image
    remosaiced_img = np.zeros((full_h, full_w, tile_c), dtype=np.uint8)

    # Place each tile onto the canvas
    for r in range(rows):
        for c in range(cols):
            if (r, c) in tile_info:
                tile_filename = tile_info[(r, c)]
                tile_path = os.path.join(PREDICTED_TILES_DIR, tile_filename)
                tile_img = cv2.imread(tile_path)

                # Ensure the tile image size matches the expected TILE_SIZE
                if tile_img.shape[:2] == (TILE_SIZE, TILE_SIZE):
                     remosaiced_img[r * TILE_SIZE:(r + 1) * TILE_SIZE, c * TILE_SIZE:(c + 1) * TILE_SIZE] = tile_img
                else:
                     print(f"Warning: Tile {tile_filename} has unexpected dimensions {tile_img.shape[:2]}. Expected ({TILE_SIZE}, {TILE_SIZE}). Skipping.")


    # Save the remosaiced image
    cv2.imwrite(REMOsaICED_OUTPUT_PATH, remosaiced_img)

    print(f"Remosaiced image saved to {REMOsaICED_OUTPUT_PATH}")

Remosaiced image saved to /content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_mosaic_1.png
